# All filters generator

### Parameters

In [2]:
#################################
# IMPORTANT SETUP
REWRITE_FILE = 0
MODIFIED = 1
#################################


FRAC = 15           # Number of fractional pixels for interpolation (eg: 1/4 of resolution)
TAPS = 6            # Number of taps in the filter
PADDING = 2         # How much padding to add in each four sides
ROUNDING_TERM = 64  # Number used to divide the sums (= sum of all coefficients in horizontal)
IN_NUM = 6          # 1 line of 6 samples (6-tap)
IN_BIT_WIDTH = 8    # Number of bits per sample
OUT_NUM = 15        # There are 15 filters used to generate 1/16 precision

#### Coefficients

In [3]:
coefficients = [
                    [ 1,  -3,  63,   4,  -2, 1],
                    [ 1,  -5,  62,   8,  -3, 1],
                    [ 2,  -8,  60,  13,  -4, 1],

                    [ 3, -10,  58,  17,  -5, 1],
                    [ 3, -11,  52,  26,  -8, 2],
                    [ 2,  -9,  47,  31, -10, 3],
                    [ 3, -11,  45,  34, -10, 3],

                    [ 3, -11,  40,  40, -11, 3],
                    [ 3, -10,  34,  45, -11, 3],
                    [ 3, -10,  31,  47,  -9, 2],
                    [ 2,  -8,  26,  52, -11, 3],

                    [ 1,  -5,  17,  58, -10, 3],
                    [ 1,  -4,  13,  60,  -8, 2],
                    [ 1,  -3,   8,  62,  -5, 1],
                    [ 1,  -2,   4,  63,  -3, 1]
                    ]

In [4]:
# Pre calculated in Google Sheets [TCC] Affine interpolation of a 4x4
coefs_sizes = [
                [8,	10,	14, 10,	10,	8],
                [8,	11,	14,	11,	10,	8],
                [9,	12,	14,	12,	11,	8],
                [10, 12, 14, 13, 11,8],
                [10, 12, 14, 13, 12,9],
                [9,	12, 14,	13,	12,	10],
                [10, 12, 14, 14, 12, 10],
                [10, 12, 14, 14, 12, 10],
                [10, 12, 14, 14, 12, 10],
                [10, 12, 13, 14, 12, 9],
                [9,	12,	13,	14,	12,	10],
                [8,	11,	13,	14,	12,	10],
                [8,	11,	12,	14,	12,	9],
                [8,	10,	11,	14,	11,	8],
                [8,	10,	10,	14,	10,	8]]

In [5]:
import math
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime 

### Variables calculation

#### Input and outputs

In [6]:
# Input size
in_width = IN_NUM * IN_BIT_WIDTH

# Input text
in_text = "\tinput signed ["+str(in_width-1)+":0] X;"

# Output ports
out_names = []
for i in range(1,OUT_NUM+1):
    if i != OUT_NUM:
        out_names.append('\tY'+str(i)+',')
    else:
        out_names.append('\tY'+str(i))
out_names_concat = '\n'.join(out_names)

# print(in_text)
# print(out_names_concat)

Obs: outputs sizes are calculated after all calculations are ready in the end of this code

In [7]:
# coefs_sizes = [[0 for x in range(TAPS-2)] for y in range(FRAC)] 
# max_input = -(2**(IN_BIT_WIDTH-1))

# for j in range(0,TAPS-2):
#     for i in range(0,FRAC):
#         coef = coefficients[i][j]
#         print(coef)
#         if (coef<0 and (math.log(abs(coef),2))%1==0):
#             # a = 1
#             coefs_sizes[i][j] = math.ceil(math.log(abs(max_input*coef),2))+1
#         else:
#             coefs_sizes[i][j] = math.ceil(math.log(abs(max_input*coef),2))+2

# # CEILING(LOG(ABS($E$19*F2),2))+1
# coefs_sizes

#### Filter outputs names

In [8]:
# To instantiate module: .Y1 (r_t0_f1_1);
filters_out = []

# To create wire: wire signed [7:0] r_t0_f1_1;
filters_wire = []

# Stores filter's output wires in the same shape of the coefficients
wire_out_filter_name = [[0 for x in range(TAPS)] for y in range(FRAC)] 

for t in range(0,TAPS):
    for y in range(1,FRAC+1):
        c = coefficients[y-1][t]
        s = coefs_sizes[y-1][t]
        if y != 15:
            if c > 0:
                wire_out_filter_name[y-1][t] = 'r_t'+str(t)+'_f'+str(y)+'_'+str(c)
            else:
                wire_out_filter_name[y-1][t] = 'r_t'+str(t)+'_f'+str(y)+'_'+str(abs(c))+'neg'
            filters_out.append('\t\t.Y'+str(y)+'\t('+str(wire_out_filter_name[y-1][t])+'),')
        else:
            if c > 0:
                wire_out_filter_name[y-1][t] = 'r_t'+str(t)+'_f'+str(y)+'_'+str(c)
            else:
                wire_out_filter_name[y-1][t] = 'r_t'+str(t)+'_f'+str(y)+'_'+str(abs(c))+'neg'
            filters_out.append('\t\t.Y'+str(y)+'\t('+str(wire_out_filter_name[y-1][t])+')')
        filters_wire.append('\twire signed ['+str(s-1)+':0]\t'+str(wire_out_filter_name[y-1][t])+';')

# To instantiate module: .Y1 (r_t0_f1_1);
filters_out_concat = '\n'.join(filters_out)

# To create wire: wire signed [7:0] r_t0_f1_1
filters_wire_concat = '\n'.join(filters_wire)

# print(filters_out_concat)
# print(filters_wire_concat)

#### First round sums

In [10]:
# wire for first sums [15][3]: s_f1_t0_t1
wire_sum_1_name = [[0 for x in range(int(TAPS/2))] for y in range(FRAC)] 
# print(len(wire_sum_1_name),len(wire_sum_1_name[0]))
# Sizes of the resulting first sums (size for wire_sum_1_name)
wire_sum_1_size = [[0 for x in range(int(TAPS/2))] for y in range(FRAC)]

# wire declaration for first sums
wire_sum_1_declaration = []

# First sums calculation
wire_sum_1_calculation = []

for f in range(1,FRAC+1):
    for t in range(0,int(TAPS/2)):
        wire_sum_1_name[f-1][t] = 's_f'+str(f)+'_t'+str(2*t)+'_t'+str(2*t+1)

        s1 = coefs_sizes[f-1][2*t]                  # Size of p1
        s2 = coefs_sizes[f-1][2*t+1]                # Size of p2
        
        name1 = wire_out_filter_name[f-1][2*t]
        name2 = wire_out_filter_name[f-1][2*t+1]
        # print(s1,s2)

        if s1 > s2:
            p1 = '\t' + str(name1)
            p2 = ' + {'

            for bit in range(0,s1-s2):
                p2 = p2+str(name2)+'['+str(s2-1)+'], '
                
            p2 = p2+str(name2)+'}; \t\t// '+str(s1)+' + '+str(s2)+' = '+str(s1+1)+' bits'
            wire_sum_1_size[f-1][t] = s1 + 1
        else:
            p1 = '\t{'
            p2 = ' + '+str(name2)+'; \t\t// '+str(s1)+' + '+str(s2)+' = '+str(s2+1)+' bits'

            for bit in range(0,s2-s1):
                p1 = p1 + str(name1)+'['+str(s1-1)+'], '

            p1 = p1+str(name1)+'}'
            wire_sum_1_size[f-1][t] = s2 + 1

        wire_sum_1_declaration.append('\twire signed ['+str(wire_sum_1_size[f-1][t]-1)+':0]\t'+str(wire_sum_1_name[f-1][t])+';\t// '+str(wire_sum_1_size[f-1][t])+' bits')
        wire_sum_1_calculation.append('\t\tassign '+str(wire_sum_1_name[f-1][t])+' = '+str(p1)+str(p2))

wire_sum_1_declaration_concat = '\n'.join(wire_sum_1_declaration)
wire_sum_1_calculation_concat = '\n'.join(wire_sum_1_calculation)

# print(wire_sum_1_declaration_concat)
# print(wire_sum_1_calculation_concat)

#### Second round sums

Essa parte só funciona para filtros de 6 taps

In [11]:
# wire for second sums [15][3]: s_f1_0
wire_sum_2_name = [[0 for x in range(2)] for y in range(FRAC)] 

# Sizes of the resulting first sums (size for wire_sum_1_name)
wire_sum_2_size = [[0 for x in range(2)] for y in range(FRAC)]

# wire declaration for second sums
wire_sum_2_declaration = []

# Second sums calculation
wire_sum_2_calculation = []

for f in range(1,FRAC+1):
    for t in range(0,2):
        wire_sum_2_name[f-1][t] = 's_f' + str(f) + '_' + str(t)

        s1 = wire_sum_1_size[f-1][2*t]   
        name1 = wire_sum_1_name[f-1][2*t]

        if t == 1:
            s2 = wire_sum_2_size[f-1][t-1]
            name2 = wire_sum_2_name[f-1][t-1]          
        else:
            s2 = wire_sum_1_size[f-1][2*t+1]   
            name2 = wire_sum_1_name[f-1][2*t+1]

        if s1 > s2:
            p1 = '\t'+str(name1)
            p2 = ' + {'
            
            for bit in range(0,s1-s2):
                p2 = p2+str(name2)+'['+str(s2-1)+'], ' 
            p2 = p2+str(name2)+'}; \t\t// '+str(s1)+' + '+str(s2)+' = '+str(s1+1)+' bits'
            
            wire_sum_1_size[f-1][t] = s1 + 1
        else:
            p1 = '\t{'
            p2 = ' + '+str(name2)+'; \t\t// '+str(s1)+' + '+str(s2)+' = '+str(s2+1)+' bits'

            for bit in range(0,s2-s1):
                p1 = p1 + str(name1)+'['+str(s1-1)+'],  '
            p1 = p1+str(name1)+'}'

            wire_sum_2_size[f-1][t] = s2 + 1

        wire_sum_2_declaration.append('\twire signed ['+str(wire_sum_2_size[f-1][t]-1)+':0]\t'+str(wire_sum_2_name[f-1][t])+';\t// '+str(wire_sum_2_size[f-1][t])+' bits')
        wire_sum_2_calculation.append('\t\tassign '+str(wire_sum_2_name[f-1][t])+' = '+str(p1)+str(p2))

wire_sum_2_declaration_concat = '\n'.join(wire_sum_2_declaration)
wire_sum_2_calculation_concat = '\n'.join(wire_sum_2_calculation)

# print(wire_sum_2_declaration_concat)
# print(wire_sum_2_calculation_concat)

#### Applying the rounding term 

In [12]:
# To store all shifts left of the final sum
wire_sum_shr_name = []

# To store the sizes of wire_sum_shr_name
wire_sum_shr_size = []

# To write the wire declaration
shr_declaration = []

# To write the calculations
shr_calculation = []

# Shifts to outputs
shr_to_out = []

# Number of left shifts
shifts = int(math.log(ROUNDING_TERM,2))

for f in range(1,FRAC+1):
    wire_sum_shr_name.append('s_f'+str(f)+'_shr')
    wire_sum_shr_size.append(int(wire_sum_2_size[f-1][1]-shifts))

    shr_declaration.append('\twire signed ['+str(wire_sum_shr_size[f-1]-1)+':0]\t'+str(wire_sum_shr_name[f-1])+';\t // '+str(wire_sum_shr_size[f-1])+' bits')
    shr_calculation.append('\t\tassign '+str(wire_sum_shr_name[f-1])+' = '+str(wire_sum_2_name[f-1][1])+' >> '+str(shifts)+';')
    shr_to_out.append('\tassign Y'+str(f)+' = '+str(wire_sum_shr_name[f-1])+';\t// '+str(wire_sum_shr_size[f-1])+' bits')

shr_declaration_concat = '\n'.join(shr_declaration)
shr_calculation_concat = '\n'.join(shr_calculation)
shr_to_out_concat = '\n'.join(shr_to_out)

# print(shr_declaration_concat)
# print(shr_calculation_concat)
print(shr_to_out_concat)

	assign Y1 = s_f1_shr;	// 11 bits
	assign Y2 = s_f2_shr;	// 11 bits
	assign Y3 = s_f3_shr;	// 11 bits
	assign Y4 = s_f4_shr;	// 11 bits
	assign Y5 = s_f5_shr;	// 11 bits
	assign Y6 = s_f6_shr;	// 11 bits
	assign Y7 = s_f7_shr;	// 11 bits
	assign Y8 = s_f8_shr;	// 11 bits
	assign Y9 = s_f9_shr;	// 11 bits
	assign Y10 = s_f10_shr;	// 11 bits
	assign Y11 = s_f11_shr;	// 11 bits
	assign Y12 = s_f12_shr;	// 11 bits
	assign Y13 = s_f13_shr;	// 11 bits
	assign Y14 = s_f14_shr;	// 11 bits
	assign Y15 = s_f15_shr;	// 11 bits


### Ouput declaration (after all calculations are ready)

In [13]:
# Output port modes
out_port = []
for i in range(1,OUT_NUM+1):
    out_port.append('\toutput signed ['+str(wire_sum_shr_size[i-1]-1)+':0] Y'+str(i)+';')
out_port_concat = '\n'.join(out_port)

print(out_port_concat)

	output signed [10:0] Y1;
	output signed [10:0] Y2;
	output signed [10:0] Y3;
	output signed [10:0] Y4;
	output signed [10:0] Y5;
	output signed [10:0] Y6;
	output signed [10:0] Y7;
	output signed [10:0] Y8;
	output signed [10:0] Y9;
	output signed [10:0] Y10;
	output signed [10:0] Y11;
	output signed [10:0] Y12;
	output signed [10:0] Y13;
	output signed [10:0] Y14;
	output signed [10:0] Y15;


### Final Verilog text

In [14]:
if MODIFIED == 1:
    modif = str(datetime.today().strftime('%d/%m/%Y'))

In [15]:
verilog = '''/*------------------------------------------------------------------------------
 * File: all_filters.v
 * Date generated: 13/02/2023
 * Date modified: '''+modif+'''
 * Author: Bruna Suemi Nagai
 * Description: Concatenating all 15 filters
 *------------------------------------------------------------------------------ */

module all_filters (
        X,
    '''+out_names_concat+'''
);


// ------------------------------------------
// Parameters
// ------------------------------------------

    // Bit width of input data 
    parameter integer IN_SIZE = 'd8;
	 
	 
// ------------------------------------------
// Port mode declarations
// ------------------------------------------

'''+in_text+'''
'''+out_port_concat+'''

// ------------------------------------------
// Wires declarations
// ------------------------------------------

//  wire signed [8:0] w2;
//  wire signed [9:0] w2_;

	 
// ------------------------------------------
// Combinational logic
// ------------------------------------------

//  assign w1 = X;
//  assign w4 = w1 << 2;
//  assign w3 = w4 - w1;
//  assign w5 = w1 + w4;
//  assign w2_ = -1 * w2;

  
// ------------------------------------------
// Outputs
// ------------------------------------------

//  assign Y1 = w2_;


endmodule //all_filters

'''
print(verilog)

/*------------------------------------------------------------------------------
 * File: all_filters.v
 * Date generated: 13/02/2023
 * Date modified: 31/03/2023
 * Author: Bruna Suemi Nagai
 * Description: Concatenating all 15 filters
 *------------------------------------------------------------------------------ */

module all_filters (
        X,
    	Y1,
	Y2,
	Y3,
	Y4,
	Y5,
	Y6,
	Y7,
	Y8,
	Y9,
	Y10,
	Y11,
	Y12,
	Y13,
	Y14,
	Y15
);


// ------------------------------------------
// Parameters
// ------------------------------------------

    // Bit width of input data 
    parameter integer IN_SIZE = 'd8;
	 
	 
// ------------------------------------------
// Port mode declarations
// ------------------------------------------

	input signed [47:0] X;
	output signed [10:0] Y1;
	output signed [10:0] Y2;
	output signed [10:0] Y3;
	output signed [10:0] Y4;
	output signed [10:0] Y5;
	output signed [10:0] Y6;
	output signed [10:0] Y7;
	output signed [10:0] Y8;
	output signed [10:0] Y9